In [1]:
import os
import re
import datetime as dt

import pandas as pd

In [2]:
import fitz # install using: pip install PyMuPDF

In [3]:
fitz.version

('1.21.1', '1.21.1', '20221213000001')

In [4]:
print(fitz.__doc__)


PyMuPDF 1.21.1: Python bindings for the MuPDF 1.21.1 library.
Version date: 2022-12-13 00:00:01.
Built for Python 3.7 on darwin (64-bit).



In [5]:
from util.util import slice_filename

In [6]:
filename_list = os.listdir('./pdf')
filename_list

['[22.10.24]_포스트 팬데믹 시대의 통화정책 조합_신한투자증권.pdf',
 '[22.10.20]_채권 Daily (10.20)_유안타증권.pdf',
 '[22.12.08]_미국채 커브에서 덜어내려는 레벨 부담_하나증권.pdf',
 '[23.01.02]_전섹터 스프레드 축소, 등급전망 하향은 증가_현대차증권.pdf',
 '[22.12.26]_아직은 우량물에 집중할 시기_신한투자증권.pdf',
 '[22.10.12]_채권 Daily (10.12)_유안타증권.pdf',
 '[22.11.28]_기차는 아직 종작역에 도착하지 않았다_교보증권.pdf',
 '[22.09.14]_환율은 금리보다 외생변수_메리츠증권.pdf',
 '[22.10.19]_채권사이클이 돌아온다 (1)_하이투자증권.pdf',
 '[22.07.26]_한국 2분기 GDP: 민간소비가 해냈지만, 일회성에 그칠_이베스트증권.pdf',
 '[22.11.03]_속도 조절이 답은 아니다_하이투자증권.pdf',
 '[23.01.27]_채권 Daily (01.27)_유안타증권.pdf',
 '[22.12.23]_신한 Bond.Zip (12월 23일)_신한투자증권.pdf',
 '[23.01.26]_Daily Bond Morning Brief(2023.01.25)_다올투자증권.pdf',
 '[23.01.19]_BOJ 통화정책 유지 여파로 금리 급락_교보증권.pdf',
 '[23.01.19]_Eugenes FICC Update(2023.1.19)_유진투자증권.pdf',
 '[22.10.31]_채권 Daily (10.31)_유안타증권.pdf',
 '[22.08.29]_잭슨 홀 심포지엄 리뷰: 연준과 싸우지 마라 (Don’t _교보증권.pdf',
 '[23.01.05]_Daily Bond Morning Brief(2023.01.05)_다올투자증권.pdf',
 '[22.12.22]_Eugenes FICC Update(2022.12.22)_유진투자증권.pdf',
 '[22.12.12]_경계해야 할 서비스 물가 경직성_신한

In [7]:
#하나증권 Test
hana_securities_list = []
for filename in filename_list:
    slice_list = slice_filename(filename)
    company_name = slice_list[2][1:-4]
    if company_name == '하나증권':
        # print(filename)
        hana_securities_list.append(filename)
hana_securities_list.sort(reverse=True)
hana_securities_list

['[23.01.26]_어둠 속에도 별은 빛나고: HY (BB+), IG (BBB-, A0)_하나증권.pdf',
 '[23.01.19]_크레딧 시장에 찾아온 강한 랠리, 그리고 해석_하나증권.pdf',
 '[23.01.19]_숏커버와 디스인플레이션_하나증권.pdf',
 '[23.01.18]_매도자(발행사) 우위시장_하나증권.pdf',
 '[23.01.17]_국고5년 적정레벨 추정_하나증권.pdf',
 '[23.01.16]_대세를 거스르는 BOJ, 뒤늦은 변화_하나증권.pdf',
 '[23.01.16]_1월 금통위 Review: 동결 같은 인상_하나증권.pdf',
 '[23.01.13]_열린 결말 속 시장은 피크아웃을 기대_하나증권.pdf',
 '[23.01.12]_[1월 금통위 Preview] 동결 조건 점검 ②_하나증권.pdf',
 '[23.01.11]_낡은 연초효과_하나증권.pdf',
 '[23.01.09]_미 국채 금리 급락: 임금상승 둔화, 서비스업 약화_하나증권.pdf',
 '[23.01.09]_[1월 금통위 Preview] 동결 조건 점검 ①_하나증권.pdf',
 '[23.01.06]_새롭게, 그리고 또 다시 등장한 변수들: ② 미국 대내_하나증권.pdf',
 '[23.01.05]_연초효과 발생시 1분기 성과 점검_하나증권.pdf',
 '[23.01.05]_새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요_하나증권.pdf',
 '[23.01.05]_미국 회사채 시장 Outlook: 발행(Issuance) & 만기(Ma_하나증권.pdf',
 '[23.01.05]_[12월 FOMC 의사록] 시장 반응에 우려 표명_하나증권.pdf',
 '[23.01.04]_인동초(忍冬草)_하나증권.pdf',
 '[22.12.29]_확률로 보는 연초효과_하나증권.pdf',
 '[22.12.23]_ESG적응(Adaptation)_하나증권.pdf',
 '[22.12.21]_연초시장 대응전략_하나증권.pdf',
 '[22.12.15]_[12월 FOMC] 신뢰를 잃은 연준_

In [8]:
# target_filename = '[23.01.05]_새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요_하나증권.pdf'
target_filename = '[23.01.04]_인동초(忍冬草)_하나증권.pdf'
# target_filename = '[23.01.18]_매도자(발행사) 우위시장_하나증권.pdf'
# target_filename = '[23.01.16]_1월 금통위 Review: 동결 같은 인상_하나증권.pdf'
# target_filename = '[23.01.11]_낡은 연초효과_하나증권.pdf'

In [9]:
doc = fitz.open('./pdf/' + target_filename)
'PowerPoint' in doc.metadata['creator']

False

In [10]:
def get_pdf_text(target_filename):
    text = ""
    with fitz.open('./pdf/' + target_filename) as doc:        
        for page in doc:
            text += page.get_text().replace('\uf09f', '')

    text = text.strip()
    return text

In [11]:
def get_pdf_text_by_page(target_filename):
    page_list = []
    text = ""
    with fitz.open('./pdf/' + target_filename) as doc:        
        for page in doc:
            text = page.get_text().strip()
            text = text.replace('\uf09f', '')
            page_list.append(text)

    return page_list

기본 분석은 '다.' 로 끝나는 문장  
향후 추가 문장 구절 추출시 종료 단어 별도 선정하여 테스트 필요  

In [20]:
def tidy_sentences(section):
    # sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    # 권고, 전망 등 문장 끝맺음말 추가 필요
    sentence_enders = re.compile(r'((?<=수준)|'
                                 r'(?<=않음)|'
                                 r'(?<=지속)| (?<=지속)\.|'
                                 r'(?<=반영)|'
                                 r'(?<=상승)|'
                                 r'(?<=하락)|'
                                 r'(?<=지속)|'
                                 r'(?<=다)\.)\s+')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', '').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [21]:
text = get_pdf_text(target_filename)
text

'인동초(忍冬草) \nIssue Comment \nCredit \n \n \nCredit 김상만, CFA \n02-3771-7508 \ncredinal@hanafn.com \n \nRA 김태현 \n02-3771-8045 \nTaehyun.kim@hanafn.com \n \nCompliance Notice \n본 조사자료는 고객의 투자에 정보를 제공할 목적으로 \n작성되었으며, 어떠한 경우에도 무단 복제 및 배포될 수 \n없습니다. 또한, 본 자료에 수록된 내용은 당사가 신뢰할 만한 \n자료 및 정보로 얻어진 것이나, 그 정확성이나 완전성을 보장할 \n수 없으므로 투자자 자신의 판단과 책임하에 최종결정을 \n하시기 바랍니다. 따라서 어떠한 경우에도 본 자료는 고객의 \n주식투자 결과에 대한 법적 책임소재의 증빙 자료로 사용될 \n수 없습니다. \n \n \n \n \n \n \n \n \n \n \n2023년 1월 4일 I Global Asset Research \nCrediVille \n여건은 아직 겨울이나 꽃은 피어날 수 있다 \n \n 다사다난했다라는 표현이 무색할 정도로 어려웠던 2022년이 지나고 새해가 밝았다. 통\n상 힘든 한해를 보내고 새해를 맞이하게 되면 뭔가 잘되지 않을까라는 기대감으로 시작\n했던 것이 일반적인 경험이었다고 친다면 2023년을 맞은 느낌은 그렇지 않은 쪽에 더 \n가깝다. 마치 방학숙제를 다하지 못한 상태에서 개학하는 심정이랄까. \n 그도 그럴 것이 작년 하반기 레고랜드 사태로 인한 신용경색위험은 결과적으로 생각보\n다 빨리 불길을 잡는데 성공하였다. 그리고 그에 더해 신용스프레드마저 철없는(?) 연\n말랠리를 시현할 정도로 시장분위기는 안정 이상의 강세를 보이고 있는 중이다. 그렇지\n만 레고랜드 사태가 우리에게 환기시켜준 부동산관련 금융불균형요인은 아직 진행중에 \n있기 때문에 마냥 랠리를 누리기에도 녹록치 않은 환경이다. \n 사실 신용스프레드는 2022년초부터 지속적으로 상승세를 보였다. 시장금리상승에 따른 \n금융비

In [22]:
page_list = get_pdf_text_by_page(target_filename)

In [23]:
section1 = page_list[0]
# section1 start 위치 찾기
pos = re.search(r'(본\s+자료는\s+고객의\s+주식투자의\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙자료로\s+사용될\s+수\s+없습니다.)|'
                r'(본\s+자료는\s+고객의\s+주식투자\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙\s+자료로\s+사용될\s+수\s+없습니다.)'
                , text, re.MULTILINE | re.IGNORECASE | re.DOTALL)
section1 = section1[pos.end(): ]
# 2023년 1월 18일 I Global Asset Research \nCrediVille \n
pos = re.search(r'\d{4}년\s+\d{1,2}월\s+\d{1,2}일 I Global Asset Research\s+', section1, re.MULTILINE)
section1 = section1[pos.end():]
section1 = section1.strip()
section1

'CrediVille \n여건은 아직 겨울이나 꽃은 피어날 수 있다 \n \n 다사다난했다라는 표현이 무색할 정도로 어려웠던 2022년이 지나고 새해가 밝았다. 통\n상 힘든 한해를 보내고 새해를 맞이하게 되면 뭔가 잘되지 않을까라는 기대감으로 시작\n했던 것이 일반적인 경험이었다고 친다면 2023년을 맞은 느낌은 그렇지 않은 쪽에 더 \n가깝다. 마치 방학숙제를 다하지 못한 상태에서 개학하는 심정이랄까. \n 그도 그럴 것이 작년 하반기 레고랜드 사태로 인한 신용경색위험은 결과적으로 생각보\n다 빨리 불길을 잡는데 성공하였다. 그리고 그에 더해 신용스프레드마저 철없는(?) 연\n말랠리를 시현할 정도로 시장분위기는 안정 이상의 강세를 보이고 있는 중이다. 그렇지\n만 레고랜드 사태가 우리에게 환기시켜준 부동산관련 금융불균형요인은 아직 진행중에 \n있기 때문에 마냥 랠리를 누리기에도 녹록치 않은 환경이다. \n 사실 신용스프레드는 2022년초부터 지속적으로 상승세를 보였다. 시장금리상승에 따른 \n금융비용의 증가가 처음에는 부담요인으로 작용하다가 시간이 지나면서 지정학적 리스\n크 증대에 따른 실적 둔화가 기업 펀더멘털에 부정적으로 작용하기 시작했다. 그러던 \n것이 레고랜드 사태를 맞으면서 부정적 여파가 증폭되는 양상으로 전개되었다. \n 하지만 부동산PF와 관련된 금융불균형 및 리스크요인은 채권시장내 연관고리가 크지는 \n않다. 직접적인 사업노출도가 있는 건설업 및 관련 대출익스포져가 확대된 비은행금융\n권에 국한된 성격이라고 봐야 할 것이다. 물론 이것이 전반적인 부동산경기의 둔화현상\n과 맞물리면서 경제주체들에게 부정적인 피드백으로 작용하고 있는 점은 인정한다손 치\n더라도 말이다.     \n 아무튼 지금 시점, 지금 (신용스프레드) 지점에서 고민스러운 점은 레고랜드 사태가 없\n었어도 부동산PF 또는 부동산경기를 둘러싼 부정적 상황전개가 유사한 강도로 신용채\n권시장에 추가적인 조정압력으로 작용할 수 있었겠는가의 여부이다. 그에 대한 판단이 \n향후 

In [24]:
section1_sentence, text_new = tidy_sentences(section1)
section1_sentence

['CrediVille 여건은 아직 겨울이나 꽃은 피어날 수 있다   다사다난했다라는 표현이 무색할 정도로 어려웠던 2022년이 지나고 새해가 밝았다.',
 '통상 힘든 한해를 보내고 새해를 맞이하게 되면 뭔가 잘되지 않을까라는 기대감으로 시작했던 것이 일반적인 경험이었다고 친다면 2023년을 맞은 느낌은 그렇지 않은 쪽에 더 가깝다.',
 '마치 방학숙제를 다하지 못한 상태에서 개학하는 심정이랄까.  그도 그럴 것이 작년 하반기 레고랜드 사태로 인한 신용경색위험은 결과적으로 생각보다 빨리 불길을 잡는데 성공하였다.',
 '그리고 그에 더해 신용스프레드마저 철없는(?) 연말랠리를 시현할 정도로 시장분위기는 안정 이상의 강세를 보이고 있는 중이다.',
 '그렇지만 레고랜드 사태가 우리에게 환기시켜준 부동산관련 금융불균형요인은 아직 진행중에 있기 때문에 마냥 랠리를 누리기에도 녹록치 않은 환경이다.',
 '사실 신용스프레드는 2022년초부터 지속적으로 상승세를 보였다.',
 '시장금리상승에 따른 금융비용의 증가가 처음에는 부담요인으로 작용하다가 시간이 지나면서 지정학적 리스크 증대에 따른 실적 둔화가 기업 펀더멘털에 부정적으로 작용하기 시작했다.',
 '그러던 것이 레고랜드 사태를 맞으면서 부정적 여파가 증폭되는 양상으로 전개되었다.',
 '하지만 부동산PF와 관련된 금융불균형 및 리스크요인은 채권시장내 연관고리가 크지는 않다.',
 '직접적인 사업노출도가 있는 건설업 및 관련 대출익스포져가 확대된 비은행금융권에 국한된 성격이라고 봐야 할 것이다.',
 '물론 이것이 전반적인 부동산경기의 둔화현상과 맞물리면서 경제주체들에게 부정적인 피드백으로 작용하고 있는 점은 인정한다손 치더라도 말이다.',
 '아무튼 지금 시점, 지금 (신용스프레드) 지점에서 고민스러운 점은 레고랜드 사태가 없었어도 부동산PF 또는 부동산경기를 둘러싼 부정적 상황전개가 유사한 강도로 신용채권시장에 추가적인 조정압력으로 작용할 수 있었겠는가의 여부이다.',
 '그에 대한 판단이 향후 시장대응에 

In [25]:
section2 = page_list[1]
# section2 start 위치 찾기
# CrediVille  Analyst 김상만 02-3771-7508      2    
pos = re.search(r'\b\w+\s+\w+\s+\w+\s+\d{2}-\d{3,4}-\d{4}\s+\d\s+'
                , section2, re.MULTILINE | re.IGNORECASE)
section2 = section2[pos.end(): ]
section2 = section2.strip()
section2

'윗물이 맑아야 아랫물도 맑다 \n \n \n그림 1. ‘21년 하반기 이후 공사채 대비 은행채 스프레드갭 추이 \n21년 중반이후 공사채-은행채간 스프\n레드 갭은 등락을 지속. 22년 이후 움\n직임은 한전채 동향이 결정적인 변수\n로 작용. 최근 갭 반전은 시장이 안\n정세를 찾아가는 과정을 반영  \n(15)\n(10)\n(5)\n0\n5\n10\n0.0\n0.5\n1.0\n1.5\n2.0\n2.5\n3.0\n3.5\n4.0\n4.5\n5.0\n5.5\n6.0\n21.7 21.8 21.9 21.1021.1121.12 22.1 22.2 22.3 22.4 22.5 22.6 22.7 22.8 22.9 22.1022.1122.12\n(bp)\n(%)\n공사채3Y vs. 은행채3Y 스프레드(우)\n공사채 AAA3Y\n은행채 AAA3Y\n \n \n자료: 인포맥스, 하나증권 \n \n \n그림 2. ‘22년 12월 중하순 국고금리 및 공사채 신용스프레드 추이 \n연말랠리의 또 다른 원인 중 하나는 \n국채금리의 상승. 공사채 스프레드는 \n국채금리 상승폭 이상의 축소 시현 \n70\n75\n80\n85\n90\n95\n100\n105\n110\n3.40\n3.45\n3.50\n3.55\n3.60\n3.65\n3.70\n3.75\n3.80\n12.14\n12.16\n12.18\n12.20\n12.22\n12.24\n12.26\n12.28\n12.30\n(bp)\n(%)\n국고 3Y\n공사채 AAA 3Y 스프레드(우)\n \n \n자료: 인포맥스, 하나금융투자'

In [26]:
section2_sentence, text_new = tidy_sentences(section2)
tmp = [sentence.split(' '*3) for sentence in section2_sentence]
section2_sentence = []
for item in tmp:
    section2_sentence += item
section2_sentence

['윗물이 맑아야 아랫물도 맑다',
 '그림 1. ‘21년 하반기 이후 공사채 대비 은행채 스프레드갭 추이 21년 중반이후 공사채-은행채간 스프레드 갭은 등락을 지속. 22년 이후 움직임은 한전채 동향이 결정적인 변수로 작용. 최근 갭 반전은 시장이 안정세를 찾아가는 과정을 반영.']

In [19]:
for sentence in section2_sentence:
    pos = re.search(r'\A\d{2,}'
                    , sentence, re.DOTALL)
    st = pos.end() if pos else -1 
    print(st)

-1
-1


In [21]:
section2_sentence_new = []
for sentence in section2_sentence:
    pos = re.search(r'\A\d{2,}'
                    , sentence, re.IGNORECASE)
    st = pos.end() if pos else -1
    if st > 0:
        continue
    section2_sentence_new.append(sentence)
section2_sentence = section2_sentence_new
section2_sentence

['윗물이 맑아야 아랫물도 맑다',
 '그림 1. ‘21년 하반기 이후 공사채 대비 은행채 스프레드갭 추이 21년 중반이후 공사채-은행채간 스프레드 갭은 등락을 지속. 22년 이후 움직임은 한전채 동향이 결정적인 변수로 작용. 최근 갭 반전은 시장이 안정세를 찾아가는 과정을 반영.']

In [22]:
section3 = page_list[2]
section3

'CrediVille \nAnalyst 김상만 02-3771-7508 \n \n \n3 \n \n크레딧스프레드 동향 및 전략 \n \n국고 장단기 금리 재역전, 회사채 발생시장 강한 수요 확인 \n단기물 위주로 강세 되돌리며 \n장단기 금리 역전 \n지난주 국내 채권 시장은 상고하저의 금리 흐름을 보였다. 주초반 연초 자금집행, 당국의 국채 \nWGBI 편입 의지 확인 등으로 금리는 하락하는 모습을 보였다. 그러나 이후 발표된 12월 \nFOMC 의사록에서 연준의 매파적인 스탠스가 재확인 되었고 이에 더해 주 후반 발표된 미국\n의 구인건수 및 ADP 민간 고용지표가 시장 예상치를 상회함에 미 금리는 단기물 위주로 상승\n했다. 국고 금리도 이와 연동해 결국 국고 3년 금리는 전주대비 3.7bp 하락한 3.69%, 10년 \n금리는 16.0bp 하락한 3.58%에 마감했다. \n회사채 발행시장 재개, \n우량물 강한 수요 확인 \n지난주 신용채권 시장은 우량물 중심 전구간에서 신용스프레드 축소흐름을 지속했다. 그 전주\n와 마찬가지로 연말 강세를 주도했던 초우량물 대비 상위등급 회사채 및 여전채의 상대적 강세\n가 이어졌다. 회사채 발행시장이 재개된 가운데 AAA~AA급 종목들의 수요예측이 진행, 전종\n목에서 매우 강한 수요가 확인되었다. 다만 회사채 유통시장의 경우 우량물 중심의 강세 거래\n가 지속되었으나 평가금리와 매매금리 간 괴리가 쉽게 좁혀지고 있지 않은 만큼 발행시장 대비 \n한산한 모습을 보였다. \n강세흐름 동참, 아직은 유효 \n섹터별 선별 실익 크지 않으나 \n등급간 차별적 접근은 여전히 유효 \n지금의 강세현상은 충분히 그 근거를 확보하고 있으나 확대된 변동성으로 인해 어느 정도의 경\n계심을 가질 필요는 있다. 경계심은 약세전환에 대한 것이라기 보다는 또 다른 변수가 발생했\n을 때 신용스프레드가 어떤 식으로 반응할 것인가에 대한 것이다. 일단 강세흐름에 적극적으로 \n동참하는 것이 아직은 유효하다. 섹터별 강세가 순환될 것이기에 선별의

In [23]:
# section3 start 위치 찾기
# CrediVille  Analyst 김상만 02-3771-7508      2    
pos = re.search(r'\b\w+\s+\w+\s+\w+\s+\d{2}-\d{3,4}-\d{4}\s+\d\s+'
                , section3, re.MULTILINE | re.IGNORECASE)
section3 = section3[pos.end():].strip()
section3_sentence, text_new = tidy_sentences(section3)
section3_sentence

['크레딧스프레드 동향 및 전략  국고 장단기 금리 재역전, 회사채 발생시장 강한 수요 확인 단기물 위주로 강세 되돌리며 장단기 금리 역전 지난주 국내 채권 시장은 상고하저의 금리 흐름을 보였다.',
 '주초반 연초 자금집행, 당국의 국채 WGBI 편입 의지 확인 등으로 금리는 하락하는 모습을 보였다.',
 '그러나 이후 발표된 12월 FOMC 의사록에서 연준의 매파적인 스탠스가 재확인 되었고 이에 더해 주 후반 발표된 미국의 구인건수 및 ADP 민간 고용지표가 시장 예상치를 상회함에 미 금리는 단기물 위주로 상승했다.',
 '국고 금리도 이와 연동해 결국 국고 3년 금리는 전주대비 3.7bp 하락한 3.69%, 10년 금리는 16.0bp 하락한 3.58%에 마감했다.',
 '회사채 발행시장 재개, 우량물 강한 수요 확인 지난주 신용채권 시장은 우량물 중심 전구간에서 신용스프레드 축소흐름을 지속했다.',
 '그 전주와 마찬가지로 연말 강세를 주도했던 초우량물 대비 상위등급 회사채 및 여전채의 상대적 강세가 이어졌다.',
 '회사채 발행시장이 재개된 가운데 AAA~AA급 종목들의 수요예측이 진행, 전종목에서 매우 강한 수요가 확인되었다.',
 '다만 회사채 유통시장의 경우 우량물 중심의 강세 거래가 지속되었으나 평가금리와 매매금리 간 괴리가 쉽게 좁혀지고 있지 않은 만큼 발행시장 대비 한산한 모습을 보였다.',
 '강세흐름 동참, 아직은 유효 섹터별 선별 실익 크지 않으나 등급간 차별적 접근은 여전히 유효 지금의 강세현상은 충분히 그 근거를 확보하고 있으나 확대된 변동성으로 인해 어느 정도의 경계심을 가질 필요는 있다.',
 '경계심은 약세전환에 대한 것이라기 보다는 또 다른 변수가 발생했을 때 신용스프레드가 어떤 식으로 반응할 것인가에 대한 것이다.',
 '일단 강세흐름에 적극적으로 동참하는 것이 아직은 유효하다.',
 '섹터별 강세가 순환될 것이기에 선별의 실익이 크지 않을 것 같으나, 등급별로는 상하위등급간 차별적인 접근은 여전히 유효할 것으로 전망한다.'

In [30]:
section_others = ""
for text in page_list[3:]:
    section_others += text
section_others

'CrediVille \nAnalyst 김상만 02-3771-7508 \n \n \n4 \n \n그림 2. 특수채 AAA 금리커브와 스프레드 변동 \n \n그림 3. 은행채 AAA 금리커브와 스프레드 변동 \n(15.0)\n(10.0)\n(5.0)\n0.0\n5.0\n10.0\n15.0\n0.0\n1.0\n2.0\n3.0\n4.0\n5.0\n6.0\n1Y\n2Y\n3Y\n5Y\n7Y\n10Y\n전주대비 스프레드 변동(우)\n특수채 AAA\n특수채 AAA(일주일 전)\n특수채 AAA(일년 전)\n(%)\n(bp)\n \n \n(15.0)\n(10.0)\n(5.0)\n0.0\n5.0\n10.0\n15.0\n0.0\n1.0\n2.0\n3.0\n4.0\n5.0\n6.0\n1Y\n2Y\n3Y\n5Y\n7Y\n10Y\n전주대비 스프레드 변동(우)\n은행채 AAA\n은행채 AAA(일주일 전)\n은행채 AAA(일년 전)\n(%)\n(bp)\n \n주: 2023.1.13 기준 \n자료: 인포맥스, 하나증권 \n \n주: 2023.1.13 기준 \n자료: 인포맥스, 하나증권 \n \n \n그림 4. 여전채 AA+ 금리커브와 스프레드 변동 \n \n그림 5. 여전채 AA- 금리커브와 스프레드 변동 \n(25.0)\n(21.0)\n(17.0)\n(13.0)\n(9.0)\n(5.0)\n(1.0)\n3.0\n7.0\n11.0\n15.0\n19.0\n23.0\n0.0\n1.0\n2.0\n3.0\n4.0\n5.0\n6.0\n7.0\n1Y\n2Y\n3Y\n5Y\n7Y\n10Y\n전주대비 스프레드 변동(우)\n여전채 AA+\n여전채 AA+(일주일 전)\n여전채 AA+(일년 전)\n(%)\n(bp)\n \n \n(25.0)\n(20.0)\n(15.0)\n(10.0)\n(5.0)\n0.0\n5.0\n10.0\n15.0\n20.0\n25.0\n0.0\n0.5\n1.0\n1.5\n2.0\n2.5\n3.0\n3.5\n4.0\n4.5\n5.0\n5.5\n6.0\n6.5\n7.0\n1Y\n2Y\n3Y\n

In [31]:
# 단순 차트만 있는 경우 패스
section_others_sentence, text_new = tidy_sentences(section_others)
section_others_sentence

['CrediVille Analyst 김상만 02-3771-7508   4  그림 2. 특수채 AAA 금리커브와 스프레드 변동  그림 3. 은행채 AAA 금리커브와 스프레드 변동 (15.0)(10.0)(5.0)0.05.010.015.00.01.02.03.04.05.06.01Y2Y3Y5Y7Y10Y전주대비 스프레드 변동(우)특수채 AAA특수채 AAA(일주일 전)특수채 AAA(일년 전)(%)(bp)  (15.0)(10.0)(5.0)0.05.010.015.00.01.02.03.04.05.06.01Y2Y3Y5Y7Y10Y전주대비 스프레드 변동(우)은행채 AAA은행채 AAA(일주일 전)은행채 AAA(일년 전)(%)(bp) 주: 2023.1.13 기준 자료: 인포맥스, 하나증권  주: 2023.1.13 기준 자료: 인포맥스, 하나증권   그림 4. 여전채 AA+ 금리커브와 스프레드 변동  그림 5. 여전채 AA- 금리커브와 스프레드 변동 (25.0)(21.0)(17.0)(13.0)(9.0)(5.0)(1.0)3.07.011.015.019.023.00.01.02.03.04.05.06.07.01Y2Y3Y5Y7Y10Y전주대비 스프레드 변동(우)여전채 AA+여전채 AA+(일주일 전)여전채 AA+(일년 전)(%)(bp)  (25.0)(20.0)(15.0)(10.0)(5.0)0.05.010.015.020.025.00.00.51.01.52.02.53.03.54.04.55.05.56.06.57.01Y2Y3Y5Y7Y10Y전주대비 스프레드 변동(우)여전채 AA-여전채 AA-(일주일 전)여전채 AA-(일년 전)(%)(bp) 주: 2023.1.13 기준 자료: 인포맥스, 하나증권  주: 2023.1.13 기준 자료: 인포맥스, 하나증권   그림 6. 회사채 AA- 금리커브와 스프레드 변동  그림 7. 회사채 A+ 금리커브와 스프레드 변동 (20.0)(15.0)(10.0)(5.0)0.05.010.015.020.00.01.02.03.04.05.06.07.01Y2Y3Y5Y7Y10Y전주대비 스프레드 변동

하나증권 보고서 분류
- 증권사 이름
- 애널리스트 이름 정보
- 커버리지


보고서 카테고리  
1. BONDView
2. CrediVille
3. Unfixed Idea
4. Credit Watch

### 하나증권 보고서 Section 추출 테스트

- CreditVille 보고서 타입 정교화

In [32]:
import pdb

docs = []

for target_filename in hana_securities_list:
    print('read file: {}'.format(target_filename))
    page_list = get_pdf_text_by_page(target_filename)
    
    doc = fitz.open('./pdf/' + target_filename)
    if 'PowerPoint' in doc.metadata['creator']:
        print("==> PPT File Format PASS!...")
        continue
    
    strdate = '20' + target_filename[1:9]
    mdate = dt.datetime.strptime(strdate, "%Y.%m.%d")
    
#     pos = re.search('Unfixed Idea', text, re.MULTILINE | re.IGNORECASE)
#     if not pos: 
#         continue
    
    section1_sentence = []
    section2_sentence = []
    section3_sentence = []
    section_others_sentence = []
    
    for page_index in range(min(4, len(page_list))):
        
        if page_index == 0:
            # section1 분리
            section1 = page_list[page_index]
            # section1 start 위치 찾기
            pos = re.search(r'(본\s+자료는\s+고객의\s+주식투자의\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙자료로\s+사용될\s+수\s+없습니다.)|'
                            r'(본\s+자료는\s+고객의\s+주식투자\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙\s+자료로\s+사용될\s+수\s+없습니다.)'
                            , section1, re.MULTILINE | re.IGNORECASE | re.DOTALL)
            section1 = section1[pos.end(): ]
            # 2023년 1월 18일 I Global Asset Research \nCrediVille \n
            section1 = section1.strip()
            section1_sentence, text_new = tidy_sentences(section1)
    
        elif page_index == 1:
            # section2 분리
            section2 = page_list[page_index]
            # section2 start 위치 찾기
            # CrediVille  Analyst 김상만 02-3771-7508      2    
            pos = re.search(r'\b\w+\s+\w+\s+\w+\s+\d{2}-\d{3,4}-\d{4}\s+\d\s+'
                            , section2, re.MULTILINE | re.IGNORECASE)
            st = pos.end() if pos else 0
            section2 = section2[st: ]
            section2 = section2.strip()
            section2_sentence_new = []
            for sentence in section2_sentence:
                pos = re.search(r'\A\d{2,}'
                                , sentence, re.IGNORECASE)
                st = pos.end() if pos else -1
                if st > 0:
                    continue
                section2_sentence_new.append(sentence)
            section2_sentence = section2_sentence_new
        
        elif page_index == 2:
            # section3 분리
            section3 = page_list[page_index]
            # section3 start 위치 찾기
            # CrediVille  Analyst 김상만 02-3771-7508      2    
            pos = re.search(r'\b\w+\s+\w+\s+\w+\s+\d{2}-\d{3,4}-\d{4}\s+\d\s+'
                            , section3, re.MULTILINE | re.IGNORECASE)
            st = pos.end() if pos else 0
            section3 = section3[st:].strip()
            section3_sentence, text_new = tidy_sentences(section3)
    
        else:
            # section 나머지 분리
            section_others = ""
            for text in page_list[3:]:
                section_others += text
            
            section_others_sentence, text_new = tidy_sentences(section_others)
            if len(section_others_sentence) >= 2:
                section_others_sentence = section_others_sentence[1:]
            else:
                section_others_sentence = []
    
    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1_sentence, section2_sentence, section3_sentence, section_others_sentence)

    doc = []
    

    if any(section_texts):
        for s, sentences in enumerate(section_texts):
            for p, text in enumerate(sentences):
                row = (target_filename, mdate, p, text)
                doc.append(row)
        print(f' ==> text processing completed, sentences_cnt: {len(doc)}')
    else:
        print('Empty!')
        
    docs += doc

read file: [23.01.26]_어둠 속에도 별은 빛나고: HY (BB+), IG (BBB-, A0)_하나증권.pdf
 ==> text processing completed, sentences_cnt: 19
read file: [23.01.19]_크레딧 시장에 찾아온 강한 랠리, 그리고 해석_하나증권.pdf
 ==> text processing completed, sentences_cnt: 21
read file: [23.01.19]_숏커버와 디스인플레이션_하나증권.pdf
 ==> text processing completed, sentences_cnt: 21
read file: [23.01.18]_매도자(발행사) 우위시장_하나증권.pdf
 ==> text processing completed, sentences_cnt: 33
read file: [23.01.17]_국고5년 적정레벨 추정_하나증권.pdf
 ==> text processing completed, sentences_cnt: 13
read file: [23.01.16]_대세를 거스르는 BOJ, 뒤늦은 변화_하나증권.pdf
 ==> text processing completed, sentences_cnt: 21
read file: [23.01.16]_1월 금통위 Review: 동결 같은 인상_하나증권.pdf
 ==> text processing completed, sentences_cnt: 20
read file: [23.01.13]_열린 결말 속 시장은 피크아웃을 기대_하나증권.pdf
 ==> text processing completed, sentences_cnt: 17
read file: [23.01.12]_[1월 금통위 Preview] 동결 조건 점검 ②_하나증권.pdf
 ==> text processing completed, sentences_cnt: 17
read file: [23.01.11]_낡은 연초효과_하나증권.pdf
 ==> text processing completed, s

In [26]:
df = pd.DataFrame(docs, columns=['filename', 'mdate', 'sid', 'text'])
df = df.dropna()
# df_minutes_path = os.path.join(minutes_path, 'minutes.csv')
# df.to_csv(df_minutes_path, encoding='utf-8', index=False)

In [27]:
df.to_excel('df_report.xlsx', encoding='utf-8', index=False)

In [28]:
mdate

datetime.datetime(2022, 7, 20, 0, 0)